## 4. Producing a Training Matrix

We want to take the sliding window approaches detailed above detecting genomic loci of interest, and move towards using those regions of interest to predict TMB as accurately as possible.

In [1]:
setwd("/Users/jacobbradley/Documents/CCG/Code")

### Packages

In [2]:
install.packages("tidyverse")
library(tidyverse)
# Data Handling

if (!requireNamespace("BiocManager"))
    install.packages("BiocManager")
BiocManager::install()
library(BiocManager)
# General bioinformatics packages

library(BSgenome)
library(BSgenome.Hsapiens.UCSC.hg19)
# Human genome build GChr37


The downloaded binary packages are in
	/var/folders/g9/9x6m35fn5ydf88vdlm9n1vg80000gn/T//RtmpdVJOGw/downloaded_packages


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required namespace: BiocManager
Bioconductor version 3.8 (BiocManager 1.30.3), R 3.5.1 (2018-07-02)
Update old packages: 'BiocManager', 'BiocParallel', 'broom', 'circlize',
  'edgeR', 'ELMER', 'ensembldb', 'foreign', 'gdsfmt', 'ggpubr', 'Gviz',
  'haven', 'IRdisplay', 'lattice', 'limma', 'MASS', 'Matrix', 'mclust', 'mgcv',
  'openssl', 'processx', 'RcppEigen', 'readr', 'repr', 'rtracklayer',
  'survival', 'units', 'VariantAnnotation'
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

  

In [3]:
source("TMB_Funcs.txt")
# My Stuff!

### Producing a training matrix

In [4]:
chromosomes <- c(1:22,"X","Y")
genome <- BSgenome.Hsapiens.UCSC.hg19

window_size <- 1000000
jump_length <- 100000

In [5]:
new_data <- read_tsv("/Volumes/CCG8/processed_data/brca_tcga/data_mutations_extended.txt")
#tcga_brca_hotspots <- genome_slide(maf_file = new_data, global_tmb_table = NULL, window_size, jump_length)
#write_tsv(path = '/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots', x = tcga_brca_hotspots)
tcga_brca_hotspots <- read_tsv('/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots', col_types = list(col_character(), col_guess(), col_guess(), col_guess()))

Warning message:
“Missing column names filled in: 'X54' [54]”Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  Start_Position = col_integer(),
  End_Position = col_integer(),
  Score = col_integer(),
  t_ref_count = col_integer(),
  t_alt_count = col_integer(),
  Protein_position = col_integer(),
  Hotspot = col_integer(),
  stop_WU = col_integer(),
  X54 = col_double(),
  tumor_vaf = col_double(),
  normal_ref_reads = col_integer(),
  start_WU = col_integer(),
  strand_WU = col_integer(),
  tumors_var_reads = col_integer(),
  normal_vaf = col_double(),
  tumor_ref_reads = col_integer(),
  normal_var_reads = col_integer()
)
See spec(...) for full column specifications.
Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“3282 parsing failures.
row # A tibble: 5 x 5 col     row col     expected  actual file                                         

In [12]:
#tcga_brca_hotspots_new <- genome_restricted_slide(maf_file = new_data, previous_run = tcga_brca_hotspots, zoom = 10, global_tmb_table = NULL, coverage = 10)
#write_tsv(path = '/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots_new', x = tcga_brca_hotspots_new)

tcga_brca_hotspots_new <- read_tsv('/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots_new', col_types = list(col_character(), col_guess(), col_guess(), col_guess()))

In [7]:
#tcga_brca_hotspots_new_new <- genome_restricted_slide(maf_file = new_data, previous_run = tcga_brca_hotspots_new, zoom = 10, global_tmb_table = NULL, coverage = 10)
#write_tsv(path = '/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots_new_new', x = tcga_brca_hotspots_new_new)

tcga_brca_hotspots_new_new <- read_tsv('/Users/jacobbradley/Documents/CCG/Data/tcga_brca_hotspots_new_new', col_types = list(col_character(), col_guess(), col_guess(), col_guess()))

In [198]:
getWindows <- function(window_run, panel_size, window_size) {
    
    working_panel <- tibble(chromosome = factor(character(), levels = c(1:22, "X", "Y")), position = vector())
    jump_length <- (window_run[2,"position"] - window_run[1, "position"])[1,1]
   
    coverage <- as.integer(window_size/jump_length)
    adding <- TRUE
    
    while (adding) {
        new_position <- window_run %>% 
            filter(significance == max(significance)) %>% 
            filter(chromosome == min(chromosome)) %>% 
            filter(position == min(position)) %>% 
            select(chromosome, position)
        
        if (nrow(working_panel %>% 
            add_row(chromosome = rep(new_position$chromosome[1], coverage), position = new_position$position[1] + jump_length*(0:(coverage-1))) %>% 
            distinct())*jump_length <= panel_size) {
            
            working_panel <- working_panel %>% 
                add_row(chromosome = rep(new_position$chromosome[1], coverage), position = new_position$position[1] + jump_length*(0:(coverage-1))) %>% 
                distinct() %>% 
                group_by(chromosome) %>% 
                arrange(position, .by_group = TRUE) %>% 
                ungroup()
            
        
            window_run <- window_run %>% 
                filter((chromosome != new_position$chromosome)|(position != new_position$position))
        }
        else {
            adding <- FALSE
        }
    }
    
    return(working_panel)
}

In [254]:
getMatrix <- function(maf_file, candidate_panel, jump_length, global_tmb_tabe = NULL) {
    
    maf_file <- abridge(maf_file, nonsilent = FALSE)
    
    if (is.null(global_tmb_table)) {
        global_tmb_table = TMB_Calc(maf_file, nonsilent = TRUE)
    }
    
    window_matrix <- global_tmb_table
    
    for (n in 1:nrow(candidate_panel)) {
        
        new_column <- maf_file %>% 
            filter(Chromosome == candidate_panel$chromosome[n], 
                   Start_Position %in% (candidate_panel$position[n] + 0:(jump_length - 1))) %>%
            group_by(Tumor_Sample_Barcode) %>% 
            summarise(Window = n())
    
        colnames(new_column) <- c("Tumor_Sample_Barcode", paste0("Window_", n))
        window_matrix <- full_join(window_matrix, new_column, by = "Tumor_Sample_Barcode")
        
    }
    
    window_matrix[is.na(window_matrix)]  <- 0
    return(window_matrix)
}

In [270]:
training_matrix <- getWindows(tcga_brca_hotspots_new_new, 10000, 10000) %>% 
    getMatrix(new_data, ., 1000)

Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”

In [275]:
training_matrix[training_matrix  %>% 
    select(starts_with("Window")) %>% 
    rowSums() > 0, "TMB"]
    

TMB
68.36667
115.90000
145.16667


In [277]:
training_matrix %>% 
    filter(TMB > 20)

Tumor_Sample_Barcode,TMB,Window_1,Window_2,Window_3,Window_4,Window_5,Window_6,Window_7,Window_8,Window_9,Window_10
TCGA-A2-A0T5-01,36.03333,0,0,0,0,0,0,0,0,0,0
TCGA-A8-A09Z-01,33.46667,0,0,0,0,0,0,0,0,0,0
TCGA-A8-A0A6-01,68.36667,0,0,0,0,0,1,0,0,0,0
TCGA-AC-A23H-01,115.90000,0,0,0,0,1,0,0,0,0,0
TCGA-AN-A046-01,145.16667,0,0,0,0,0,1,0,0,0,1
TCGA-AN-A0AK-01,30.80000,0,0,0,0,0,0,0,0,0,0
TCGA-AO-A128-01,22.86667,0,0,0,0,0,0,0,0,0,0
TCGA-BH-A18G-01,35.30000,0,0,0,0,0,0,0,0,0,0
TCGA-D8-A1JA-01,22.93333,0,0,0,0,0,0,0,0,0,0
TCGA-D8-A1XK-01,23.90000,0,0,0,0,0,0,0,0,0,0


In [268]:
training_matrix[which((training_matrix %>% 
    select(starts_with("Window")) %>% 
    rowSums())>1),"TMB"]

TMB
68.36667
115.90000
145.16667
24.16667


In [259]:
windows <- paste0("Window_", 1:(ncol(training_matrix)-2))
formula <- as.formula(paste("TMB ~", paste(windows, collapse = "+")))
tcga_brca_fit <- lm(formula, training_matrix)

In [260]:
tcga_brca_fit


Call:
lm(formula = formula, data = training_matrix)

Coefficients:
(Intercept)     Window_1     Window_2     Window_3     Window_4     Window_5  
     1.6634           NA           NA           NA           NA           NA  
   Window_6     Window_7     Window_8     Window_9    Window_10    Window_11  
         NA           NA           NA     103.6957     114.7901           NA  
  Window_12    Window_13    Window_14    Window_15    Window_16    Window_17  
         NA           NA      28.5967           NA           NA           NA  
  Window_18    Window_19    Window_20    Window_21    Window_22    Window_23  
         NA           NA           NA           NA           NA           NA  
  Window_24    Window_25    Window_26    Window_27    Window_28    Window_29  
    -0.3300           NA       7.6366       8.3033      16.6366       1.6533  
  Window_30    Window_31    Window_32    Window_33    Window_34    Window_35  
    21.1800       0.9144      10.0811           NA           NA